# DC-GAN (Deep Convolutional Generative Adversarial Network)

I recently opened Tensorflow's tutorials on GAN (Generative Adversarial Network), and found this [link](https://www.tensorflow.org/tutorials/generative/dcgan). 

Basically the tutorial is making a generative handwritten digits image using Keras API. But in here, I'm gonna using TensorLayerX's API to build the GAN. Some parts may not following the Tensorflow's tutorial directly. 

Things to install:
- tensorflow
- tensorlayerx
- torch (Just if you need it's backend. If not, you can skip torch)
- matplotlib
- numpy
- pillow

The goal here is to make a generative model that generates new handwritten digit images. 

In [1]:
import tensorflow;
from tensorflow.keras.datasets import mnist;
import numpy;

In [2]:
import torch;

torch.cuda.is_available()

True

## Dataset Loading & Pre-Processing

MNIST dataset is a database of handwritten digits for training. So people will type in 0 to 9 handwrittenly, taking the photo of it, and resizing it to 28 x 28 greyscale pixels.

Tensorflow Keras API had an API to download MNIST dataset, and using it directly as train-test splits. If I'm not mistaken, TensorLayerX also had one, but I prefered this one since I once use it.

By applying `.shape`, we can then see that by default, the training set had 60K worth of data while test set had 10K. From these 10K set, I split the test set into 50:50, making validation set from it.

| Splits | Total data |
|---|---|
| Train | 60000 |
| Test | 5000 |
| Val | 5000 |

In [3]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data();

# Split the test and val by 50:50
test_val_images_split = numpy.array_split(test_images, 2);
test_val_labels_split = numpy.array_split(test_labels, 2);

test_images = test_val_images_split[0];
test_labels = test_val_labels_split[0];

val_images = test_val_images_split[1];
val_labels = test_val_labels_split[1];

train_images.shape, test_images.shape, val_images.shape

((60000, 28, 28), (5000, 28, 28), (5000, 28, 28))

## Data Preparation

With the done of the data preprocessing, I will do Data Preparation using TensorLayerX's API. This part is crucial since we can't just set the data into the model without reshaping it into proper tensor format.

As mentioned above, we know that the images consists of data dimension below:

| Attribute | Size |
|---|---|
| Width | 28px |
| Height | 28px |
| Color Channels | Gray Scale |

With this information above, we can conclude that the tensor dimension should be (28 -> Width, 28 -> Height, 1 -> Color Channel). Which is, not same as the loaded version that was (28, 28).

The data preparation is done using Data Loading pattern. This pattern will encapsulating the process of loading and preprocessing data. Since the rest of data pre-processing phase below will manipulating the nature of the data, like converting it to some data format, it is best to encapsulating it into a class. 

So what we will do in this part are:

1. Set Tensorlayerx's backend as Tensorflow / Torch
2. Create Data Loader class
3. Converts every value within features with shape of (28, 28, 1) and as float32 tensor since it is easier to model to read.
4. Standardize the data within features by dividing it to 255.
5. Register all train-test-val data with Data Loader

In [4]:
import tensorlayerx;
from tensorlayerx import expand_dims, convert_to_tensor, float32, squeeze;
from tensorlayerx.dataflow import IterableDataset;

import os;

# Set Tensorlayerx's backend as Tensorflow / Torch
# os.environ["TL_BACKEND"] = "tensorflow"; # Uncomment this line to use Tensorflow's Backend
os.environ["TL_BACKEND"] = "torch"; # Uncomment this line to use Torch's backend

Using TensorFlow backend.
c:\Users\yosua\anaconda3\envs\tlx-obor\Lib\site-packages\tensorlayerx\__init__.py:45: UserWarning: The version of the backend you have installed does not match the specified backend version and may not work, please install version tensorflow 2.4.0.
  warnings.warn("The version of the backend you have installed does not match the specified backend version "


In [5]:
# Create Data Loader class
class DatasetLoader(IterableDataset):
    def __init__(self, feature, label):

        # Converts every value within features with shape of (28, 28, 1) and as float32 tensor since it is easier to model to read.
        # Standardize the data within features by dividing it to 255.
        self.data = feature.reshape((len(feature), 28, 28, 1)).astype('float32') / 255;
        self.label = label.astype('int32');
    
        print(f"Data Shape: {self.data.shape}");
        print(f"Label Shape: {self.label.shape}");

    def __getitem__(self, index):
        data = self.data[index];
        label = self.label[index];

        return data, label;

    def __len__(self):
        return len(self.data);

    def __iter__(self):
        for i in range(len(self.data)):
            yield self.data[i], self.label[i];

In [6]:
# Register all train-test-val data with Data Loader

train_set = DatasetLoader(train_images, train_labels);
test_set = DatasetLoader(test_images, train_labels);
val_set = DatasetLoader(val_images, train_labels);

Data Shape: (60000, 28, 28, 1)
Label Shape: (60000,)
Data Shape: (5000, 28, 28, 1)
Label Shape: (60000,)
Data Shape: (5000, 28, 28, 1)
Label Shape: (60000,)


In [7]:
data, label = val_set.__getitem__(0);

data.shape, label

((28, 28, 1), 5)

## Model Architecture

GAN architecture consist of 2 Models. Generator, and Discriminator. 

Discriminator model 

In [13]:
from tensorlayerx.nn import Module, Conv2d, Linear, BatchNorm, MaxPool2d, Flatten;
from tensorlayerx import LeakyReLU;
from tensorlayerx.metrics import Accuracy;
from tensorlayerx.losses import binary_cross_entropy;

In [11]:
# Generator Model

class MNIST_Model_G(Module):

    def __init__(self):
        super(MNIST_Model_G, self).__init__();

        # I don't know why this can't be from tlx.initializers import TruncatedNormal
        w_init = tlx.initializers.TruncatedNormal(stddev = 0.02);
        b_init = tlx.initializers.TruncatedNormal(mean = 1.0, stddev = 0.02);

        self.input = Input(shape = (64, 28, 28, 1));
        self.conv1 = Conv2d(out_channels = 64, kernel_size = (3, 3), act = LeakyReLU, padding = "SAME", W_init = w_init, b_init = b_init, data_format = "channel_first", name = "conv1");
        self.conv2 = Conv2d(out_channels = 64, kernel_size = (3, 3), act = LeakyReLU, padding = "SAME", W_init = w_init, b_init = b_init, data_format = "channel_first", name = "conv2");
        
        self.output = Conv2d(out_channels = 1, kernel_size = (3, 3), act = LeakyReLU, padding = "SAME", W_init = w_init, b_init = b_init, data_format = "channel_first", name = "output");

    def forward(self, x):
        x = self.conv1(x);
        x = self.conv2(x);
        x = self.output(x);

        return x;

    def construct(self, x):
        x = self.input(x);
        x = self.conv1(x);
        x = self.conv2(x);
        out = self.output(x);

        return out;

class G_With_Loss(Module):
    def __init__(self, network: Module, loss_fn):
        super(G_With_Loss, self).__init__();

        self.network = network;
        self.loss_function = loss_fn;

    def forward(self, data, ground_truth):
        predictions = self.network(data);
        loss = self.loss_function(predictions, ground_truth);

        return loss;

G_network = MNIST_Model_G();
G_netW_Loss = G_With_loss(network = G_network)

In [ ]:
class MNIST_Model_D(Module):
    
    def __init__(self):
        super(MNIST_Model_D, self).__init__();

        # I don't know why this can't be from tlx.initializers import TruncatedNormal
        w_init = tlx.initializers.TruncatedNormal(stddev = 0.02);
        b_init = tlx.initializers.TruncatedNormal(mean = 1.0, stddev = 0.02);

        self.input = Input(shape = (28, 28, 1));
        
        self.conv1 = Conv2d(out_channels = 64, kernel_size = (3, 3), act = LeakyReLU, padding = "SAME", W_init = w_init, b_init = b_init, data_format = "channel_first", name = "conv1");
        self.pool1 = MaxPool2d(kernel_size = (2, 2), name = "pool1");

        self.conv2 = Conv2d(out_channels = 32, kernel_size = (3, 3), act = LeakyReLU, padding = "SAME", W_init = w_init, b_init = b_init, data_format = "channel_first", name = "conv2");
        self.pool2 = MaxPool2d(kernel_size = (2, 2), name = "pool2");

        self.flat = Flatten(name = "flat");
        self.output = Linear(
            out_features = 1,
            W_init = tlx.initalizers.TruncatedNormal(stddev = 5-e2),
            b_init = tlx.initalizers.TruncatedNormal(mean = 1, stddev = 2e-2),
            act = LeakyReLU,
            name = "output"
        );

    def forward(self, x):
        x = self.conv1(x);
        x = self.pool1(x);
        
        x = self.conv2(x);
        x = self.pool2(x);

        x = self.flat(x);
        x = self.output(x);

        return x;

    def construct(self, x):
        x = self.input(x);

        x = self.conv1(x);
        x = self.pool1(x);
        
        x = self.conv2(x);
        x = self.pool2(x);

        x = self.flat(x);
        out = self.output(x);

        return out;

class D_With_Loss(Module):
    def __init__(self, network: Module, loss_fn):
        super(D_With_Loss, self).__init__();

        self.network = network;
        self.loss_function = loss_fn;

    def forward(self, data, ground_truth):
        predictions = self.network(data);
        loss = self.loss_function(predictions, ground_truth);

        return loss;
        